Runs the FIR model fits for the models defined in workflow #8 [8_define-FIR-models](http://localhost:8888/notebooks/8_define-FIR-models.ipynb).

Note: **you need to have run the function redef_FIRs.m, located on scratch, on the outputs of workflow #8, prior to running this notebook**

In [1]:
from bids.layout import BIDSLayout 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html
#
# set spm path to copy with use edit of spm_fMRI_design.m [see: https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=spm;f525092b.1508]
# scratch/qbi/uqkgarn1
matlab_cmd = '/scratch/qbi/uqkgarn1/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

201229-21:12:19,299 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()

fitfirs = pe.Workflow(name="fitfirs") # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


# grab and sink data

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                            outfields=['spm']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'spm': [['sub','TR']]}
dg.inputs.field_template = {'spm': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-%s/TR%s/FLGLM/SPM.mat'}

In [4]:
# debugging
dg.inputs.sub = '01'
dg.inputs.TR = '1510'
res = dg.run()
res.outputs

201229-21:12:31,242 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpmsc1cupb/data-grabber".
201229-21:12:31,246 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
201229-21:12:31,252 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



spm = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR1510/FLGLM/SPM.mat

In [5]:
inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                   name='info')
inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
inf.iterables = [('sub', ['01']), ('TR', ['1510'])]
inf.iterables 

[('sub', ['01']), ('TR', ['1510'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0]
    TR = [t for t in l if re.search('TR([0-9]*)',t)][-1]
    name = [name, TR, "FLGLM"]
    name = '/'.join(name)
    return name

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_gzip_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# estimate model

In [8]:
# EstimateModel - estimate the parameters of the model
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

# zip files

from .nii -> .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs

In [10]:
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

# Connect workflow

In [11]:
fitfirs.connect([(inf, dg, [('sub',  'sub')]),
                 (inf, dg, [('TR', 'TR')]),
                 (dg, ds, [(('spm', printSubPath), 
                            'container')]),
                 (dg, estimate, [('spm', 'spm_mat_file')]),
                 (estimate, gzip, [('beta_images', 'in_file')]),
                 (gzip, ds, [('out_file', 'PARAMS')])
])

In [ ]:
fitfirs.run()

201229-21:12:31,415 nipype.workflow INFO:
	 Workflow fitfirs settings: ['check', 'execution', 'logging', 'monitoring']
201229-21:12:31,426 nipype.workflow INFO:
	 Running serially.
201229-21:12:31,426 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.data-grabber" in "/tmp/tmpmsc1cupb/data-grabber".
201229-21:12:31,431 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
201229-21:12:31,435 nipype.workflow INFO:
	 [Node] Finished "fitfirs.data-grabber".
201229-21:12:31,436 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.estimate" in "/tmp/tmp8t3s451z/fitfirs/_TR_1510_sub_01/estimate".
201229-21:12:31,467 nipype.workflow INFO:
	 [Node] Running "estimate" ("nipype.interfaces.spm.model.EstimateModel")
